# RNN (based on Andrej Karpathy educational video)

# Let's write a backprop with tensors

In [6]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

%matplotlib inline

In [8]:
words = open('./data/names.txt', 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [9]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [11]:
block_size = 3
def build_dataset(words):
    X, Y = [], []

    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
    
    X = torch.tensor(X)
    Y = torch.tensor(Y)

    return X, Y

import random

random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])


In [12]:
# utility function we will use later when comparing manual gradients to PyTorch gradients

def cmp(s, dt, t):
    ex = torch.all(dt == t.grad).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt - t.grad).abs().max().item()
    print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [21]:
n_embd = 10
n_hidden = 64

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embd), generator=g)

W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3) / ((n_embd * block_size) ** 0.5)
b1 = torch.randn(n_hidden, generator=g) * 0.1

W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1
b2 = torch.randn(vocab_size, generator=g) * 0.1

bngain = torch.randn((1, n_hidden)) * 0.1 + 1.0
bnbias = torch.randn((1, n_hidden)) * 0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True

4137


In [22]:
batch_size = 32
n = batch_size # a shorter variable also, for convinience

ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]

In [23]:
# forward pass

emb = C[Xb]
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation
# Batchnorm Layer
bnmeani = 1 / n * hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1 / (n - 1) * (bndiff2).sum(0, keepdim=True) # note: Bessel's correction (div by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-linearity
h = torch.tanh(hpreact)
# Linear layer 2
logits = h @ W2 + b2 # output layer
# Cross-entropy loss (same as F.cross_entropy)
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# Pytorch backward pass
for p in parameters:
    p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, norm_logits, logit_maxes, logits, h, hpreact,
    bnraw, bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani, embcat, emb]:
    t.retain_grad()

loss.backward()
loss

tensor(3.4811, grad_fn=<NegBackward0>)

In [24]:
logprobs.shape

torch.Size([32, 27])

In [25]:
logprobs[range(n), Yb]

tensor([-3.6402, -3.7457, -2.9490, -3.6503, -3.1279, -3.1100, -3.4688, -2.9617,
        -2.8462, -3.8254, -3.2296, -3.5369, -3.0599, -3.3885, -4.1351, -4.2632,
        -2.9271, -3.0369, -4.7090, -3.6269, -3.7753, -2.8079, -3.4454, -3.9531,
        -3.4072, -3.1958, -3.9245, -3.3962, -3.2505, -4.3302, -2.5854, -4.0841],
       grad_fn=<IndexBackward0>)

In [ ]:
# loss = -(a + b + c) / 3 = -1/3a + -1/3b + -1/3c
# dloss/da = -1/3
# or in general dloss/da = -1/n

In [26]:
# loss = -logprobs[range(n), Yb].mean()

dlogprobs = None ## this will hold the derivative of the loss with respect to all the elements of logprobs. 
# It should be the size of logprobs.shape

dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1.0 / n

In [27]:
# let's check that this is correct

cmp('logprobs', dlogprobs, logprobs)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0


In [28]:
# logprobs = probs.log()
# d/dx log(x) = 1 / x

dprobs = (1.0 / probs) * dlogprobs # chain rule

In [29]:
cmp('probs', dprobs, probs)

probs           | exact: True  | approximate: True  | maxdiff: 0.0


In [39]:
# probs = counts * counts_sum_inv   ### these matrcies have different shapes but there is an implicit broadcasting
# d/dx ax = a

dcounts_sum_inv = (counts * dprobs).sum(1, keepdim=True)


In [40]:
dcounts_sum_inv.shape

torch.Size([32, 1])

In [41]:
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)

counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0


In [50]:
# probs = counts * counts_sum_inv

dcounts = counts_sum_inv * dprobs

In [52]:
# counts_sum_inv = counts_sum**-1

dcounts_sum = -1 * counts_sum**(-2) * dcounts_sum_inv

In [53]:
cmp('counts_sum', dcounts_sum, counts_sum)

counts_sun      | exact: True  | approximate: True  | maxdiff: 0.0


In [54]:
# counts_sum = counts.sum(1, keepdims=True)

dcounts += torch.ones_like(counts) * dcounts_sum

In [55]:
cmp('dcounts', dcounts, counts)

dcounts         | exact: True  | approximate: True  | maxdiff: 0.0


In [58]:
# counts = norm_logits.exp()

dnorm_logits = counts * dcounts

In [59]:
cmp('dnorm_logits', dnorm_logits, norm_logits)

dnorm_logits    | exact: True  | approximate: True  | maxdiff: 0.0


In [60]:
# norm_logits = logits - logit_maxes

dlogits = 1 * dnorm_logits

In [61]:
cmp('dlogits', dlogits, logits)

dlogits         | exact: False | approximate: True  | maxdiff: 6.984919309616089e-09


In [ ]:
# norm_logits = logits - logit_maxes

dlogit_maxes = 1 * dnorm_logits